In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import torch

from data._main import get_dataset
from omegaconf import OmegaConf
from utils import NEURONS_302

In [3]:
dataset_config = OmegaConf.load("../../../configs/submodule/dataset.yaml")
dataset_config.dataset.train.name = ['Kato2015', 'Nichols2017']
dataset_config.dataset.train.num_worms = 10
# Print config
print(OmegaConf.to_yaml(dataset_config))

dataset:
  train:
    name:
    - Kato2015
    - Nichols2017
    num_named_neurons: all
    num_worms: 10
    save: false
  predict:
    name: Kato2015
    num_named_neurons: 1
    num_worms: 1
    save: false



In [4]:
combined_dataset, dataset_info, neuron_counts = get_dataset(dataset_config=dataset_config.dataset.train)

In [5]:
dataset_info

,dataset,original_index,combined_dataset_index,neurons
0,Kato2015,worm0,worm0,"[AFDL, AIBL, AIBR, ALA, ASKL, ASKR, AVAL, AVAR..."
1,Kato2015,worm2,worm2,"[AIBL, AIBR, ALA, AS1, AVAL, AVAR, AVBL, AVBR,..."
2,Kato2015,worm3,worm3,"[AIBL, AIBR, ALA, AQR, ASKR, AVAL, AVAR, AVBL,..."
3,Nichols2017,worm4,worm16,"[AFDL, AIBL, AIBR, ALA, ASKL, ASKR, AVAL, AVAR..."
4,Nichols2017,worm14,worm26,"[AIBL, AIBR, ALA, AQR, AVAL, AVAR, AVEL, AVFL,..."
5,Nichols2017,worm16,worm28,"[AIBL, AIBR, ALA, AVAL, AVAR, AVBL, AVBR, AVEL..."
6,Nichols2017,worm29,worm41,"[AFDR, AIBL, AIBR, ALA, AQR, AUAL, AUAR, AVAL,..."
7,Nichols2017,worm34,worm46,"[AIBL, AIBR, ALA, AQR, ASKL, AVAL, AVAR, AVBR,..."
8,Nichols2017,worm37,worm49,"[AIBL, AIBR, ALA, AQR, ASGL, ASGR, ASKL, ASKR,..."
9,Nichols2017,worm43,worm55,"[AIBL, AIBR, ALA, AQR, ASKL, ASKR, AUAR, AVAL,..."


In [6]:
neuron_counts

,neuron,count
0,AIBL,10
1,AIBR,10
2,ALA,10
3,AVAL,10
4,AVAR,10
...,...,...
79,SIADL,1
80,SIADR,1
81,SIBDL,1
82,SMBVL,1


In [83]:
dataset_info = {
    'dataset': [],
    'original_index': [],
    'combined_dataset_index': [],
    'neurons': [],
}

combined_dataset_neurons = []

for worm, data in combined_dataset.items():
    dataset_info['dataset'].append(data['dataset'])
    dataset_info['original_index'].append(data['original_worm'])
    dataset_info['combined_dataset_index'].append(data['worm'])
    worm_neurons = [neuron for slot, neuron in data['slot_to_named_neuron'].items()]
    dataset_info['neurons'].append(worm_neurons)
    combined_dataset_neurons = combined_dataset_neurons + worm_neurons

dataset_info = pd.DataFrame(dataset_info)

combined_dataset_neurons = np.array(combined_dataset_neurons)
# Count occurernces of each neuron
neuron_counts = np.unique(combined_dataset_neurons, return_counts=True)
# Sort by neuron count
neuron_counts = np.array(sorted(zip(*neuron_counts), key=lambda x: x[1], reverse=True))
# Create a dataframe
neuron_counts = pd.DataFrame(neuron_counts, columns=["neuron", "count"])
neuron_counts

,neuron,count
0,AIBL,10
1,AIBR,10
2,AVAL,10
3,RIML,10
4,RIMR,10
...,...,...
70,SABVR,1
71,SMBDL,1
72,SMBDR,1
73,SMBVL,1
